In [2]:
import json
from requests_html import HTMLSession, HTML
session = HTMLSession()
from selenium import webdriver
browser = webdriver.Chrome(executable_path='./chromedriver/chromedriver')

1. Get link_products

In [ ]:
browser = webdriver.Chrome(executable_path='./chromedriver/chromedriver')

browser.get('https://www.amazon.com/s?i=specialty-aps&bbn=16225007011&rh=n%3A16225007011%2Cn%3A13896617011&ref=nav_em_T1_0_4_NaN_3__nav_desktop_sa_intl_computers_tablets')

url = 'https://www.amazon.com/s?i=specialty-aps&bbn=16225007011&rh=n%3A16225007011%2Cn%3A13896617011&ref=nav_em_T1_0_4_NaN_3__nav_desktop_sa_intl_computers_tablets'

f = open("dataset/amazon_product_links.txt",'w')
count = 0
while True:
    print(url)
    count += 1
    print(count)
    browser.get(url)
    post = HTML(html=browser.page_source)
    head_ = post.find('.s-include-content-margin')
    for head in head_:
        pos = head.find('.a-text-normal')
        if len(pos) != 0:
            url = 'https://www.amazon.com/' + pos[0].attrs['href'] + '\n'
            f.write(url)

    check_next = post.find('.a-last')
    if len(check_next) == 1:
        url = 'https://www.amazon.com/' + check_next[0].find('a')[0].attrs['href']       
    else:
        break

f.close()

2. Get attributes

In [ ]:
def getAttributes():
    res = ['Price']
    
    browser.get('https://www.amazon.com/s?i=specialty-aps&bbn=16225007011&rh=n%3A16225007011%2Cn%3A13896617011&ref=nav_em_T1_0_4_NaN_3__nav_desktop_sa_intl_computers_tablets')
    
    f = open('dataset/amazon_product_links.txt', 'r')
    links = f.readlines()
    f.close()

    count = 0
    for link in links:
        count += 1
        url = link[:-1]
        url = url.split('\t')

        if (len(url) == 1):
            browser.get(url[0])
            post = HTML(html=browser.page_source)
            
            specifications = post.find('#productDetails_techSpec_section_1')
            if len(specifications) != 0:
                specifications_components = specifications[0].find('tr')
                for component in specifications_components:
                    data = component.text.split('\n')
                    if data[0] not in res:
                        res.append(data[0])
            else:
                continue
            specifications = post.find('#productDetails_techSpec_section_2')
            if len(specifications) != 0:
                specifications_components = specifications[0].find('tr')
                for component in specifications_components:
                    data = component.text.split('\n')
                    if data[0] not in res:
                        res.append(data[0])
            else:
                continue
        print(count)
        if count % 100 == 0:
            print(res)
    return res

browser = webdriver.Chrome(executable_path='./chromedriver/chromedriver')
list_ = getAttributes()
dic = dict(zip(list_, [""]*len(list_)))
r = json.dumps(dic)
f = open('dataset/amazon_attrs.json', 'w')
f.write(r)
f.close()

3. Crawl data

In [15]:
class Laptop:
    def __init__(self, attrbs_list):
        self.data = dict(zip(attrbs_list, [""]*len(attrbs_list)))
        
    def convertDataToJsonStr(self):
        return json.dumps(self.data)

    def convertDataToStrCSV(self):
        return '\t'.join(self.data.values()) + '\n'
    
    def get_keys_list(self):
        return list(self.data.keys())
          
    def setValue(self, key, value):
        self.data[key] = value

In [16]:
with open('dataset/amazon_attrs.json', 'r') as f:
    attrs = f.read()
attrbs_list = list(json.loads(attrs).keys())

In [ ]:
browser = webdriver.Chrome(executable_path='./chromedriver/chromedriver')

browser.get('https://www.amazon.com/s?i=specialty-aps&bbn=16225007011&rh=n%3A16225007011%2Cn%3A13896617011&ref=nav_em_T1_0_4_NaN_3__nav_desktop_sa_intl_computers_tablets')

f = open('dataset/amazon_product_links.txt', 'r')
links = f.readlines()
f.close()

g = open('dataset/amazon_product_infos.csv', 'w')
g1 = open('dataset/amazon_product_infos.txt', 'w')

temp = Laptop(attrbs_list)
title = '\t'.join(temp.get_keys_list()) + '\n'
g.write(title)

count = 0
for link in links:
    count += 1

    url = link[:-1]
    url = url.split('\t')
    
    print(count)
    print(url)

    if (len(url) == 1):
        laptop = Laptop(attrbs_list)
        
        browser.get(url[0])
        post = HTML(html=browser.page_source)

        specifications = post.find('#olp-upd-new')
        if len(specifications) != 0:
            specifications = specifications[0].find('.a-color-price')
            if len(specifications) != 0:
                laptop.setValue('Price', specifications[0].text)
            else:
                continue
        else:
            continue


        specifications = post.find('#productDetails_techSpec_section_1')
        if len(specifications) != 0:
            specifications_components = specifications[0].find('tr')
            for component in specifications_components:
                data = component.text.split('\n')
                if len(data) == 2:
                    laptop.setValue(data[0], data[1])
        else:
            continue
        specifications = post.find('#productDetails_techSpec_section_2')
        if len(specifications) != 0:
            specifications_components = specifications[0].find('tr')
            for component in specifications_components:
                data = component.text.split('\n')
                if len(data) == 2:
                    laptop.setValue(data[0], data[1])
        else:
            continue
    
    g.write(laptop.convertDataToStrCSV())
    g1.write(laptop.convertDataToStrCSV())
    
g.close()
g1.close()

109
['https://www.amazon.com//Acer-A114-32-C1YA-Celeron-Personal-Windows/dp/B07D97S1CR/ref=sr_1_109?qid=1575099992&s=computers-intl-ship&sr=1-109']
110
['https://www.amazon.com//Samsung-Galaxy-Tablet-SM-T280NZKMXAR-Warranty/dp/B077NPX4VQ/ref=sr_1_110?qid=1575099992&s=computers-intl-ship&sr=1-110']
111
['https://www.amazon.com//Microsoft-Surface-Pro-Intel-Core/dp/B07K4FMSS8/ref=sr_1_111?qid=1575099992&s=computers-intl-ship&sr=1-111']
112
['https://www.amazon.com//HP-14-Inch-Chromebook-Laptop-Renewed/dp/B07VQM9YMX/ref=sr_1_112?qid=1575099992&s=computers-intl-ship&sr=1-112']
113
['https://www.amazon.com//Samsung-Cellular-Unlocked-SM-T295-International/dp/B07XC3CW4G/ref=sr_1_113?qid=1575099994&s=computers-intl-ship&sr=1-113']
114
['https://www.amazon.com//Acer-Display-i7-8565U-GeForce-A515-54G-73WC/dp/B07QKWHWC9/ref=sr_1_114?qid=1575099994&s=computers-intl-ship&sr=1-114']
115
['https://www.amazon.com//Laptop-Storage-Memory-Camera-Windows/dp/B07Y49YVVG/ref=sr_1_115?qid=1575099994&s=computer